In [ ]:
import numpy as np

def read_data_from_file(filename):
    with open(filename, 'r') as file:
        m = int(file.readline())
        n = int(file.readline())
        l = list(map(int, file.readline().split()))
        s = list(map(int, file.readline().split()))
        matrix = [list(map(int, file.readline().split())) for _ in range(n+1)]
    return m, n, l, s, matrix


def write_dzn_file(filename, m, n, l, s, matrix,low_bound,up_bound,d_low_bound, min_l,max_l,ratio_packages,symmetry):
    with open(filename, "w") as f:
        f.write(f"courier = {m};\n")
        f.write(f"items = {n};\n")
        f.write(f"low_bound = {low_bound};\n")
        f.write(f"up_bound = {up_bound};\n")
        f.write(f"min_l = {min_l};\n")
        f.write(f"max_load = {max_l};\n")
        f.write(f"d_low_bound = {d_low_bound};\n")
        f.write(f"ratio_packages = {ratio_packages};\n")
        f.write(f"symmetry = {symmetry};\n")
        f.write("\n")       
        f.write(f"courier_size = {str(l)};\n")
        f.write(f"item_size = {str(s)};\n")
        f.write("\n")
        f.write("distances = [| ")
        
        for row in matrix:
            f.write(", ".join([str(val) for val in row]))
            f.write("\n     |")
        f.write("];\n")
        

def set_lower_bound(distances):
    """
    :param distances: matrix of distances
    :result lb: the lower bound for the objective funciton
    :result dist_lb: the lower bound for the array of courier distances
    """
    last_row = distances[-1]
   # last_column = [distances[i][-1] for i in range(len(distances[0]))]
    last_column = distances[:,-1]
    value1 = last_column[np.argmax(last_row)] + max(last_row)
    value2 = last_row[np.argmax(last_column)] + max(last_column)
    lb = max(value1, value2)

    return lb

number_of_instances = [str(numero).zfill(2) for numero in range(1, 22)]
count = []
for i in number_of_instances:
    filename = "input/"+"inst"+i+".dat"
    m, n, l, s, matrix = read_data_from_file(filename)
    l = list(np.sort(l)[::-1])
    matrix = np.array(matrix)
    #matrix, s = sort_matrix(matrix,s)
    sub_tour = (True if min(l) >= max(s) else False)
    low_bound = set_lower_bound(matrix)
    d_low_bound = int(np.min(matrix[matrix.nonzero()]) * (n/m +1))
    up_bound = sum([max(matrix[i]) for i in range(n)])
    up_bound = np.max(matrix)*n //m + np.max(np.array(matrix)[:,-1])
    min_l = np.min(s)*n // m
    max_l = np.sum(np.partition(s, m)[m-1:]) if np.sum(np.partition(s, m)[m:]) <= max(l) else max(l)
    ratio_packages = np.max(l)//np.min(l)
    print(np.sum(matrix))
    file_write = "Data/data"+ i +".dzn"
    write_dzn_file(file_write, m, n, l, s, matrix,low_bound,
                   up_bound,d_low_bound, min_l,max_l,ratio_packages,int(np.array_equal(matrix,matrix.T)))
print("number of symmetric instancies",count)






In [ ]:
def sort_matrix(distance_matrix,places_weights):
    n = distance_matrix.shape[1] - 1
    new_vector = np.argsort(places_weights)[::-1] 
    # Get indices for sorting in decreasing order

    # Sort places_weights and distance_matrix accordingly
    distance_column = distance_matrix[new_vector,-1]
    distance_row = distance_matrix[-1,new_vector]
    distance_column = np.append(distance_column, 0)
    distance_row = np.append(distance_row, 0)

    sorted_places_weights = [places_weights[i] for i in new_vector]
    new_vector = np.append(new_vector, n)
    sorted_distance_matrix = distance_matrix[new_vector, :][:, new_vector]
    sorted_distance_matrix[-1,:] = distance_row
    sorted_distance_matrix[:,-1] = distance_column
    return sorted_distance_matrix,sorted_places_weights

In [ ]:
import numpy as np

class Mcpp_var:
     
     def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D
        self.low_dist_bound = set_lower_bound()

     


def load_instances(num: int):
  if num < 10:
    num = "0"+str(num)
  else:
    num = str(num)
  
  file = open(f"input/inst{num}.dat", 'r')
  
  m = int(file.readline())
  n = int(file.readline())
  l = [int(x) for x in file.readline().split(" ") if x!= ""]
  s = [int(x) for x in file.readline().split(" ") if x!= ""]
  D = []
  for i in range(n+1):
      D.append([int(x) for x in file.readline().split(" ") if x!= "\n" if x!= ""])
  
  return Mcpp_var(m, n, l, s, D) 

In [17]:
import numpy as np

n = 10
m = 9
vector = np.array([5, 3, 2, 8, 2, 7, 1, 4, 6, 9, 10])

result = np.sum(np.partition(vector, m+1)[m+1:])
print("Risultato:", result)

Risultato: 10


In [2]:
# Da modificare 

from minizinc import Instance, Model, Solver


class data_instances:
    def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D

data = data_instances(m,n,l,s,matrix)



def execute_minizinc_model(model_path, data):
    # Load the MiniZinc model
    model = Model(model_path)
    # Create a MiniZinc solver instance
    gecode  = Solver.lookup("chuffed")
    
    inst = Instance(gecode, model)
    
    inst["m"] = data.m
    inst["n"] = data.n
    inst["l"] = data.l
    inst["s"] = data.s
    inst["D"] = data.D
    
    
    sol = inst.solve().solution
    
    return sol

model_path = "Model2.mzn"
#execute_minizinc_model(model_path,data)

ModuleNotFoundError: No module named 'minizinc'